In [2]:
import time
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.animation as animation
from itertools import combinations

%matplotlib qt

# Vecino más Cercano para el TSP

In [3]:
def nearest_neighbor(Edges, weights):
    """
        Edges: Arreglo de aristas del grafo
        weights: distancias de cada arista
    """
    # Encontrar la arista de menor peso para iniciar el recorrido
    next_edge = np.argmin(weights)
    
    T = list(Edges[next_edge, :])
    ordered_el = [Edges[next_edge]]
    S = weights[next_edge]

    E = np.delete(Edges, next_edge, axis=0)
    d = np.delete(weights, next_edge, axis=0)


    while len(E) > 0:
        # Encontrar minima distancia conectada a un extremo del recorrido
        next_edge = np.argmin(np.where(np.any(np.isin(E, [T[0], T[-1]]), axis=1), d, np.inf))

        # Revisar si la arista va al principio o al final del recorrido
        a, b = E[next_edge]
        if a == T[0]: 
            T.insert(0, b)
        elif b == T[0]:
            T.insert(0, a)
            b, a = a, b
        elif a == T[-1]:
            T.append(b)
        else:
            T.append(a)
            b, a = a, b
        ordered_el.append(E[next_edge])
        S += d[next_edge]

        # Descartar aristas invalidas para el problema
        mask = ~np.logical_or(np.any(E == a, axis=1), np.all(np.isin(E, T), axis=1))
        E = E[mask]
        d = d[mask]

    # Cerrar el recorrido
    a, b = np.sort([T[0], T[-1]])
    ordered_el.append([a, b])
    
    for edg, dist in zip(Edges, weights):
        if edg[0] == a and edg[1] == b:
            S += dist

    return T, S, ordered_el

def edges_from_tour(T):
    return [(x, y) for x, y in zip(T, T[1:])] + [(T[-1], T[0])]

## Solución para n particular

In [18]:
n = 10
C = np.loadtxt("data/datos_unicos.txt", max_rows=n)
E = np.array(list(combinations(range(n), 2)))
d = np.linalg.norm(C[E[:, 0], :] - C[E[:, 1], :], axis=1)

t = time.perf_counter()
T, S, ordered_el = nearest_neighbor(E, d)
t = time.perf_counter() - t
print("Numero de ciudades:\t{}\nCosto:\t\t\t{}\nTiempo:\t\t\t{} s".format(n, S, t))

Numero de ciudades:	10
Costo:			710.3022594871238
Tiempo:			0.0017053999999916414


In [33]:
G = nx.Graph()
G.add_weighted_edges_from(zip(E[:, 0], E[:, 1], d))

locs = dict(zip(range(n), C[:n, :]))
edgs = edges_from_tour(T)

nx.draw_networkx(G, locs, edgelist=edgs, node_size=200)
plt.show()

In [6]:
fig, ax = plt.subplots(figsize=(10, 10))
nx.draw_networkx(G, locs, edgelist=[], node_size=200, ax=ax)
def anim_tour(i):
    nx.draw_networkx_edges(G, locs, [ordered_el[i]], ax=ax)
#ax.set_aspect('equal')
anim = animation.FuncAnimation(fig, anim_tour, range(len(edgs)), interval=1000)
plt.show()

## Comparación contra solución exacta

In [11]:
# N = [5, 10, 20, 30, 40, 50, 75, 100, 125, 150, 200, 250, 300, 400, 500, 600, 634]
# bench_NN = np.zeros([len(N), 3])

# for n, row in zip(N, bench_NN):
#    C = np.loadtxt("data/datos_unicos.txt", max_rows=n)
#    E = np.array(list(combinations(range(n), 2)))
#    d = np.linalg.norm(C[E[:, 0], :] - C[E[:, 1], :], axis=1)

#    t = time.perf_counter()
#    _, S, oredered_el = nearest_neighbor(E, d)
#    t = time.perf_counter() - t
#    row[:] =[n, t, S] 

# np.savetxt("data/benchmark_NN.txt", bench_NN)

In [28]:
bench_NN = np.loadtxt("data/benchmark_NN.txt")
bench_dantzig = np.loadtxt("data/benchmark_dantzig.txt")

plt.plot(bench_dantzig[:, 0], bench_dantzig[:, 1])

plt.axis([5, 300, 0, 400])
plt.ylabel("Tiempo [s]")
plt.xlabel("Número de Ciudades")
plt.title("Tiempo de Ejecución con PLE")

plt.savefig("img/res_d.png", dpi=300)
plt.show()

In [29]:
plt.plot(bench_NN[:, 0], bench_NN[:, 1])

plt.axis([5, 634, 0, 10])
plt.ylabel("Tiempo [s]")
plt.xlabel("Número de Ciudades")
plt.title("Tiempo de Ejecución con NN")

plt.savefig("img/res_nn.png", dpi=300)
plt.show()

In [31]:
plt.plot(bench_dantzig[:, 0], bench_dantzig[:, 1] - bench_NN[:13, 1])

plt.axis([5, 300, 0, 400])
plt.ylabel("Tiempo [s]")
plt.xlabel("Número de Ciudades")
plt.title("Diferencia en Tiempo de Ejecución")

plt.savefig("img/diff_tiempo.png", dpi=300)
plt.show()

In [32]:
plt.plot(bench_dantzig[:, 0], bench_dantzig[:, 2], label="PLE")
plt.plot(bench_NN[:13, 0], bench_NN[:13, 2], label="Vecino más Cercano")

plt.axis([5, 300, 0, 7000])
plt.ylabel("Costo")
plt.xlabel("Número de Ciudades")
plt.title("Costo de Solución")
plt.legend()

plt.savefig("img/cto_d_nn.png", dpi=300)
plt.show()